In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from scipy.stats import shapiro
# imputer
from sklearn.impute import SimpleImputer, KNNImputer

url1 ='https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom.data'
names = ["feature" + str(x) for x in range(1, 591)]
df1 = pd.read_csv(url1,sep=" ", names=names, na_values = "NaN",header=None)
df1.head()

url2 ='https://archive.ics.uci.edu/ml/machine-learning-databases/secom/secom_labels.data'
df2 = pd.read_csv(url2,sep=" ",names = ["Result","Date"])

#df2.columns =['Pass/Fail','Date']
df2.head()

#Convertion of Date into Datetime from Object(String) data types
df2['Date'] = pd.to_datetime(df2['Date'])
df2.dtypes

#Joinig TWO df1 and df2 Dataframe naming SECOM
Secom = pd.concat([df1,df2],axis = 1)
print(Secom)

Secom = Secom.drop(['Date'],axis=1)
                   
# establish target and features of the manufacturing data
# set the target to the encoded manufacturing outcome column
y = Secom[['Result']]
# set the features as the rest of the dataset after dropping the features that are no
x = Secom.drop(['Result'], axis=1)

# getting the shapes of new data sets x and y
print("shape of x:", x.shape)
print("shape of y:", y.shape)

#Splitting data

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 1,stratify = y)

# getting the counts
print("shape of x_train: ", x_train.shape)
print("shape of x_test: ", x_test.shape)
print("shape of y_train: ", y_train.shape)
print("shape of y_test: ", y_test.shape)

#Removing features having Missing ratio more than 50%

def percentna(dataframe, threshold):
    columns = dataframe.columns[(dataframe.isnull().sum()/len(dataframe))>threshold]
    return columns.tolist()

na_columns = percentna(x_train, 0.5)
len(na_columns)
x_train_dn = x_train.drop(na_columns, axis=1)
x_train_dn.shape

#Low Variance Filter
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(x_train_dn)

constant_columns = [column for column in x_train_dn.columns
                    if column not in x_train_dn.columns[var_thres.get_support()]]

print(len(constant_columns))

x_train_lv = x_train_dn.drop(constant_columns,axis=1)

      feature1  feature2   feature3   feature4  feature5  feature6  feature7  \
0      3030.93   2564.00  2187.7333  1411.1265    1.3602     100.0   97.6133   
1      3095.78   2465.14  2230.4222  1463.6606    0.8294     100.0  102.3433   
2      2932.61   2559.94  2186.4111  1698.0172    1.5102     100.0   95.4878   
3      2988.72   2479.90  2199.0333   909.7926    1.3204     100.0  104.2367   
4      3032.24   2502.87  2233.3667  1326.5200    1.5334     100.0  100.3967   
...        ...       ...        ...        ...       ...       ...       ...   
1562   2899.41   2464.36  2179.7333  3085.3781    1.4843     100.0   82.2467   
1563   3052.31   2522.55  2198.5667  1124.6595    0.8763     100.0   98.4689   
1564   2978.81   2379.78  2206.3000  1110.4967    0.8236     100.0   99.4122   
1565   2894.92   2532.01  2177.0333  1183.7287    1.5726     100.0   98.7978   
1566   2944.92   2450.76  2195.4444  2914.1792    1.5978     100.0   85.1011   

      feature8  feature9  feature10  ..

In [4]:
x_test_drop = x_test.copy()

x_test_drop = x_test_drop.drop(na_columns, axis=1)

x_test_drop = x_test_drop.drop(constant_columns,axis=1)

#### Outlier imputation

In [5]:
def outliers(feat):
 upper_limit = feat.mean() + 3*feat.std()
 lower_limit = feat.mean() - 3*feat.std()

 feat = np.where(
    feat >upper_limit,
    upper_limit,
    np.where(
       feat <lower_limit,
        lower_limit,
        feat ))
 return feat

x_train_outliers_imputation =x_train_lv.copy()
for column in x_train_outliers_imputation:
  x_train_outliers_imputation[column] = outliers(x_train_outliers_imputation[column])

#### Mean imputation

In [6]:
numColumns = x_train_outliers_imputation.select_dtypes(include=np.number).columns.tolist();

# initialize imputer. use strategy='mean' for mean imputation
imputer = SimpleImputer(strategy='mean')# fit the imputer on X_train. we pass only numeric columns with NA's here.
imputer.fit(x_train_outliers_imputation[numColumns])# transform the data using the fitted imputer
X_train_mean_impute = imputer.transform(x_train_outliers_imputation[numColumns])
X_test_mean_impute = imputer.transform(x_test_drop[numColumns])# put the output into DataFrame. remember to pass columns used in fit/transform
X_train_mean_impute = pd.DataFrame(X_train_mean_impute, columns=numColumns)
X_test_mean_impute = pd.DataFrame(X_test_mean_impute, columns=numColumns)

## Implement Boruta feature selection

In [7]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=1)
boruta_selector.fit(np.array(X_train_mean_impute), np.array(y_train)) 

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	450
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	450
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	450
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	450
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	450
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	450
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	450
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	17
Rejected: 	433
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	17
Rejected: 	433
Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	17
Rejected: 	433
Iteration: 	11 / 100
Confirmed: 	0
Tentative: 	17
Rejected: 	433
Iteration: 	12 / 100
Confirmed: 	3
Tentative: 	14
Rejected: 	433
Iteration: 	13 / 100
Confirmed: 	3
Tentative: 	14
Rejected: 	433
Iteration: 	14 / 100
Confirmed: 	3
Tentative: 	14
Rejected: 	433
Iteration: 	15 / 100
Confirmed: 	3
Tentative: 	14
Rejected: 	433
Iteration: 	16 / 100
Confirmed: 	6
Tentat

BorutaPy(estimator=RandomForestClassifier(max_depth=5, n_estimators=105,
                                          random_state=RandomState(MT19937) at 0x1E5D4517D40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1E5D4517D40, verbose=2)

In [8]:
# Ranking of Boruta

print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_) 

Ranking:  [ 79 370 151 280 239 290 415 315 173 184  90 149  92 276  19 216 206 411
 212   6 127 391 379  65  31  23 226 391 207 399 202 133 335 215 269 184
   9 286   2  79 258 273 177 407 169 147 399  38 337 262  54 343 157 329
   1 293  19  49  12   1   1 144  96 144  61   5 188 155 435 126  96  72
 237 270 197  95 244 106 338 325 393 319  98 224 425 291 196 426 424 394
 418  66 263 428 146  52 108 321 192 295 350 136 237 111 217 317  17 378
  83 153  25 306  36  39 358 229 302 308 387 341 284  24 277 404 318 226
 233 244 382 249 373 338 348 371 193 129 124 175 333  47   1  89 282  21
  45 149 402 388  42  26  85 399 166 134 125  13 179 293 340 365 372 325
 409 303 383 375 203 222 210 239  54 139 163 120 249 138  43 144 232  22
 237   4 435 210 172 435  37 140  58 266  32 188 156 204 300 361 252 102
 220 312 164 235 131 224 197 119  40  41 241 129 374 363 395 365   1 268
 180 406 272 191 221 309 380 408 412 174 350 288 165 176 183 288  56   3
 114 278  74  32 140 368 389   7  17  79 

In [9]:
selected_rf_features = pd.DataFrame({'Feature':list(X_train_mean_impute.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features.sort_values(by='Ranking').head(30)

,Feature,Ranking
330,feature427,1
403,feature540,1
369,feature476,1
345,feature442,1
426,feature563,1
281,feature349,1
214,feature268,1
140,feature154,1
54,feature60,1
274,feature342,1


In [10]:
# Using the BorutaPy object to transform the features in the dataset.

X_Filtered_train = boruta_selector.transform(np.array(X_train_mean_impute))

X_Filtered_test = boruta_selector.transform(np.array(X_test_mean_impute)) 


#### Acuracy check

In [11]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import scorer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV

classifiers = [['RandomForest :',RandomForestClassifier()]]

for name,classifier in classifiers:
    clf=classifier.fit(X_Filtered_train,y_train)
    y_pred=classifier.predict(X_Filtered_test)
    print(f'\n {name} \n')
    print(f'Training Score for {name}  {clf.score(X_Filtered_train,y_train) * 100:.2f}' )
    print(f'Testing Score for {name} {clf.score(X_Filtered_test,y_test) * 100:.2f}' )
    print(f'Classification report  \n {classification_report(y_test,y_pred)}' )
    print(f'Confusion matrix  \n {confusion_matrix(y_test,y_pred)}' )
    print(f'ROC AUC  : {roc_auc_score(y_test,y_pred)}' )


 RandomForest : 

Training Score for RandomForest :  100.00
Testing Score for RandomForest : 92.99
Classification report  
               precision    recall  f1-score   support

          -1       0.94      0.99      0.96       440
           1       0.25      0.03      0.06        31

    accuracy                           0.93       471
   macro avg       0.59      0.51      0.51       471
weighted avg       0.89      0.93      0.90       471

Confusion matrix  
 [[437   3]
 [ 30   1]]
ROC AUC  : 0.5127199413489736
